In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.insert(0, "..")

In [ ]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [ ]:
!ls ../pretrained/neural_fields/

In [ ]:
log_path = "../pretrained/neural_fields/jolteon"

In [ ]:
import yaml
import argparse
import os.path as osp
from train import dict2namespace
from pprint import pprint
from utils import load_imf

original, cfg = load_imf(
    log_path, 
    config_fpath=osp.join(log_path, "config.yaml"), 
    return_cfg=True)
original

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
import skimage
import skimage.measure
from trainers.utils.vis_utils import imf2mesh
    
original.eval()
res = 256
new_mesh = imf2mesh(original, res=res, threshold=0.00, normalize=True, norm_type='res')
new_mesh

In [ ]:
# verts = (new_mesh.vertices * 2 - res) / float(res)
# new_mesh = trimesh.Trimesh(vertices=verts, faces=new_mesh.faces)
print(new_mesh.vertices.max(), new_mesh.vertices.min())
mp.plot(new_mesh.vertices, new_mesh.faces)

# Creating Handles

1. Creating a set of deforming handles (i.e. sparse points on the surface that will change according to the shape)
2. Creating a set of statisc handles (i.e. sparse points on the surface that will remain unchanged)

In [ ]:
verts, faces = new_mesh.vertices, new_mesh.faces
print(verts.max(axis=0))
print(verts.min(axis=0))

In [ ]:
from scipy.spatial.transform import Rotation as ROT

# neg_vert = np.concatenate([
#     verts[
#         np.logical_and(verts[:, 1] < -0.65, verts[:, 2] > 0), :],
#     verts[
#         np.logical_and(verts[:, 1] < -0.65, np.logical_and(verts[:, 2] < 0, verts[:, 0] < 0)), :]
# ], axis=0)
# pos_vert = verts[
#     np.logical_and(verts[:, 1] < -0.65, np.logical_and(verts[:, 2] < 0, verts[:, 0] > 0)), 
#     :]

neg_vert = verts[np.logical_and(verts[:, 1] < -0.65, verts[:, 2] > 0), :]
pos_vert = verts[np.logical_and(verts[:, 1] < -0.65, verts[:, 2] < 0), :]

# Rotate around the a center, with certain eular angle
rot_degree = -45
rot_axis = 'x'
rot_mat = ROT.from_euler(rot_axis, rot_degree, degrees=True).as_matrix()
# center = np.array([0, -0.35, -0.05]).reshape(1, 3)
center = np.array([0, -0., -0.0]).reshape(1, 3)
translate = np.array([0., 0.0, -0.1]).reshape(1, 3)
pos_vert_transf = np.matmul(pos_vert - center, rot_mat) + center + translate

p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(neg_vert, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(pos_vert, shading={"point_color": "red", "point_size": 0.05})
p.add_points(center, shading={"point_color": "black", "point_size": 0.3})
p.add_points(pos_vert_transf, shading={"point_color": "green", "point_size": 0.05})

# Compare to Open3D ASAP Algorithm

In [ ]:
from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh

handles = np.concatenate([neg_vert, pos_vert], axis=0)
targets = np.concatenate([neg_vert, pos_vert_transf], axis=0)

mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
                                 steps=1000, smoothed_alpha=1.)


In [ ]:
p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
p.add_points(neg_vert, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(pos_vert, shading={"point_color": "red", "point_size": 0.05})
p.add_points(pos_vert_transf, shading={"point_color": "green", "point_size": 0.05})

In [ ]:
mesh_prime_o3d = trimesh_to_o3dmesh(mesh_prime_tri)
# o3d_pcl = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(targets))
# mesh_prime_o3d.compute_vertex_normals()
# mesh_prime_o3d.paint_uniform_color([0.1, 0.5, 0.1])
# o3d_mesh0 = trimesh_to_o3dmesh(new_mesh)
# o3d_mesh0.paint_uniform_color([0.1, 0.1, 0.5])
# o3d.visualization.draw_geometries([mesh_prime_o3d, o3d_mesh0, o3d_pcl])

In [ ]:
# # Saving it to data
# out_dir = "../data/jolteon/deform_%s_rotate%d.npy" % (rot_axis, rot_degree)
# np.save(out_dir, {
#     "handles": handles,
#     "targets": targets,
#     'gtr_verts': mesh_prime_tri.vertices,
#     'gtr_faces': mesh_prime_tri.faces
# })
# out_dir